In [1]:
# 导入相关的库（主要就是imblearn库）
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score
from numpy import mean
 
# 导入数据
df = pd.read_csv(r'./data/bank-additional/bank-additional-full.csv', ';') # '';'' 为分隔符
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [2]:
df['y'].value_counts()/len(df)

no     0.887346
yes    0.112654
Name: y, dtype: float64

In [3]:
# 只保留数值型变量（简单操作）
df = df.loc[:,
['age', 'duration', 'campaign', 'pdays',
       'previous', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed','y']]
# target由 yes/no 转为 0/1
df['y'] = df['y'].apply(lambda x: 1 if x=='yes' else 0)
df['y'].value_counts()

0    36548
1     4640
Name: y, dtype: int64

In [5]:
# 1、随机欠采样的实现
# 导入相关的方法
from imblearn.under_sampling import RandomUnderSampler
 
# 划分因变量和自变量
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
 
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.40)
 
# 统计当前的类别占比情况
print("Before undersampling: ", Counter(y_train))
 
# 调用方法进行欠采样
undersample = RandomUnderSampler(sampling_strategy='majority')
 
# 获得欠采样后的样本
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)
 
# 统计欠采样后的类别占比情况
print("After undersampling: ", Counter(y_train_under))
 
# 调用支持向量机算法 SVC
model=SVC()
 
clf = model.fit(X_train, y_train)
pred = clf.predict(X_test)
print("ROC AUC score for original data: ", roc_auc_score(y_test, pred))
 
clf_under = model.fit(X_train_under, y_train_under)
pred_under = clf_under.predict(X_test)
print("ROC AUC score for undersampled data: ", roc_auc_score(y_test, pred_under))

Before undersampling:  Counter({0: 22008, 1: 2704})
After undersampling:  Counter({0: 2704, 1: 2704})
ROC AUC score for original data:  0.5988740095717713
ROC AUC score for undersampled data:  0.8306554588652563


In [6]:
%%time
# 2、使用SMOTE进行过采样
# 导入相关的方法
from imblearn.over_sampling import SMOTE
 
# 划分因变量和自变量
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
 
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.40)
 
# 统计当前的类别占比情况
print("Before oversampling: ", Counter(y_train))
 
# 调用方法进行过采样
SMOTE = SMOTE()
 
# 获得过采样后的样本
X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)
 
# 统计过采样后的类别占比情况
print("After oversampling: ",Counter(y_train_SMOTE))
 
# 调用支持向量机算法 SVC
model=SVC()
 
clf = model.fit(X_train, y_train)
pred = clf.predict(X_test)
print("ROC AUC score for original data: ", roc_auc_score(y_test, pred))
 
clf_SMOTE= model.fit(X_train_SMOTE, y_train_SMOTE)
pred_SMOTE = clf_SMOTE.predict(X_test)
print("ROC AUC score for oversampling data: ", roc_auc_score(y_test, pred_SMOTE))

Before oversampling:  Counter({0: 21949, 1: 2763})
After oversampling:  Counter({0: 21949, 1: 21949})
ROC AUC score for original data:  0.5973857033945625
ROC AUC score for oversampling data:  0.8522838921357143


In [7]:
%%time
#  3、欠采样和过采样的结合（使用pipeline）
# 导入相关的方法
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
 
# 划分因变量和自变量
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
 
#  定义管道
model = SVC()
over = SMOTE(sampling_strategy=0.4)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under), ('model', model)]
pipeline = Pipeline(steps=steps)
 
# 评估效果
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=5, n_jobs=-1)
score = mean(scores)
print('ROC AUC score for the combined sampling method: %.3f' % score)

ROC AUC score for the combined sampling method: 0.937


In [8]:
%%time
# 4、如何获取最佳的采样率？
# 导入相关的方法
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
 
# 划分因变量和自变量
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
 
# values to evaluate
over_values = [0.3,0.4,0.5]
under_values = [0.7,0.6,0.5]
for o in over_values:
  for u in under_values:
    # define pipeline
    model = SVC()
    over = SMOTE(sampling_strategy=o)
    under = RandomUnderSampler(sampling_strategy=u)
    steps = [('over', over), ('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=5, n_jobs=-1)
    score = mean(scores)
    print('SMOTE oversampling rate:%.1f, Random undersampling rate:%.1f , Mean ROC AUC: %.3f' % (o, u, score))

SMOTE oversampling rate:0.3, Random undersampling rate:0.7 , Mean ROC AUC: 0.937
SMOTE oversampling rate:0.3, Random undersampling rate:0.6 , Mean ROC AUC: 0.937
SMOTE oversampling rate:0.3, Random undersampling rate:0.5 , Mean ROC AUC: 0.937
SMOTE oversampling rate:0.4, Random undersampling rate:0.7 , Mean ROC AUC: 0.938
SMOTE oversampling rate:0.4, Random undersampling rate:0.6 , Mean ROC AUC: 0.937
SMOTE oversampling rate:0.4, Random undersampling rate:0.5 , Mean ROC AUC: 0.938
SMOTE oversampling rate:0.5, Random undersampling rate:0.7 , Mean ROC AUC: 0.938
SMOTE oversampling rate:0.5, Random undersampling rate:0.6 , Mean ROC AUC: 0.938
SMOTE oversampling rate:0.5, Random undersampling rate:0.5 , Mean ROC AUC: 0.938
Wall time: 15min 4s
